In [1]:
import pandas as pd
import numpy as np
import rasterio
import shutil
import sys
import os

from osgeo import gdal
from io import StringIO
from itertools import chain

import fileinput
import subprocess
import xml.etree.ElementTree as ET
from configparser import ConfigParser, ExtendedInterpolation

In [5]:
path = input("Directory of WASI files: ")
print (path)

Directory of WASI files: H:\Sentinel2_fit
H:\Sentinel2_fit


In [6]:
dictionary = {}
for root, dirs, files in os.walk (path):
    for name in files:
        if name.endswith((".fit")) and not name.startswith(('.')):
            #print ('dat file:', name)
            #print ('Root file', root)
            #print (root+'/'+name)
            
            raster = gdal.Open(root+'/'+name)
            
            if raster is None:
                print ('Unable to Open Raster Data')
            
            stat = []
            for b in range(raster.RasterCount+1)[1::]:
                band = raster.GetRasterBand(b)
                arry = band.ReadAsArray().astype(np.float32)
                mask = arry == 0
                arry = np.ma.masked_array(arry, mask)
                Min = np.min(arry)
                Max = np.max(arry)
                Mean = np.mean(arry)
                Median = np.ma.median(arry)
                Count = np.ma.count(arry)
                stat.append([b, Min, Max, Mean, Median, Count])
            dictionary[name] = stat


In [7]:
with open('WASI_statistics.txt', 'w') as file:
    file.write('\n{:}\n'.format('Statistics'))
    for name in dictionary.keys():
        file.write('\n{:}\n'.format(name))
        file.write('{0[0]}\t{0[1]}\t{0[2]}\t{0[3]}\t{0[4]}\t{0[5]}\n'.format(['BAND#','MINIMUM','MAXIMUM', 'MEAN', 'MEDIAN', 'COUNT']))
        for stat in dictionary[name]:
            file.write("{0[0]}\t{0[1]}\t{0[2]}\t{0[3]}\t{0[4]}\t{0[5]}\n".format(stat))
    file.close() 